In [3]:
import json
import pandas as pd
import os

In [18]:
filename_in = 'out_gen/out_gen.json'
filename_out = 'out_gen/out_gen_edited.json'

with open(filename_in) as fin:
    with open(filename_out, 'w+') as fout:
        for line in fin:
            fout.write(line.replace('\'', '\"'))

In [24]:
filename_out = 'out_gen/out_gen_edited.json'
with open(filename_out) as fin:
    data = json.load(fin)

In [29]:
key = list(data.keys())[0]
key

'((TOP (S (ADVP (RB Next) (PP (IN to) (NP (DT the) (NNP Yippee) (NNP Noodle) (NNP Bar)))) (NP (EX there)) (VP (VBZ is) (NP (NP (DT a) (ADJP (RB moderately) (JJ good)) (NN restaurant)) (VP (VBN called) (S (NP (NNP Alimentum)))))) (. .) (. \n))),)'

In [30]:
values = data[key]
len(values)

640

In [40]:
sequence = [v for i, v in enumerate(values) if i % 64 == 5]
for s in sequence:
    print('----------------------------------------------')
    print(s)

----------------------------------------------
average UNK near an a food , Waterman low 5 The of . Golden The is UNK is range more has a Bells customer , a food a food UNK the a friendly . serves average UNK The is range prices UNK food customer is UNK . is The a food UNK UNK UNK food 5 and Â£ in , cheap UNK The a
----------------------------------------------
, UNK a 5 The Wrestlers , START low 5 rating UNK . of The is UNK END The UNK is a a Golden UNK a friendly UNK a Strada the UNK food Located a riverside is UNK Waterman range UNK UNK food customer is UNK a is UNK a food UNK UNK Golden Golden 5 customer UNK the , UNK UNK UNK Indian
----------------------------------------------
Boat Â£ the 5 CafÃ© Punter . START low 5 rating and . The The , 
 END START UNK is a friendly friendly UNK a friendly customer START UNK the range , . The Riverside a . CafÃ© 5 UNK UNK food are is UNK a , UNK UNK Â£ UNK UNK Cambridge food 5 rating UNK are . UNK UNK food UNK
---------------------------------

In [41]:
sequence = [v for i, v in enumerate(values) if i % 64 == 6]
for s in sequence:
    print('----------------------------------------------')
    print(s)

----------------------------------------------
. END high food river CafÃ© friendly The low 5 UNK less UNK . is The UNK END is and UNK UNK Waterman Â£ UNK . of food Cambridge of UNK UNK is The , UNK centre a Located food and , is a friendly rating the UNK , The food . 5 a END 5 food are The UNK friendly UNK is are
----------------------------------------------
a are food food centre CafÃ© friendly START low 5 UNK less UNK Cambridge is The UNK END is , UNK UNK UNK Â£ UNK . is food START of UNK average is the are UNK range a Located the CafÃ© , is UNK UNK rating the UNK Boat a English a Â£ a the Hall are food rating UNK Golden UNK . a
----------------------------------------------
a Hall food 5 END Adriatic a Centre low 5 UNK less UNK , is The UNK END is Boat UNK a UNK UNK UNK . is END START The customer Boat is The is friendly food a Golden END a the is UNK UNK UNK are UNK friendly START Strada a customer a are the are food UNK UNK Golden The a UNK
--------------------------------------

In [61]:
sequence = [v for i, v in enumerate(values) if i % 64 == 8]
for s in sequence:
    print('----------------------------------------------')
    print(s)

----------------------------------------------
is friendly the 5 range It UNK , low 5 It friendly . of are The of 5 of friendly are are is UNK food suited a Golden the UNK UNK Golden Cambridge UNK The the UNK The Bibimbap food START UNK the are 5 a low a UNK a a a friendly . 5 the are food average the is Fitzbillies END friendly
----------------------------------------------
the friendly the 5 CafÃ© It , Grove low 5 is friendly . Mill The The of END of friendly are a END UNK , food a Indian the UNK UNK provides Boat UNK The the range . UNK . START UNK the Golden The UNK UNK I friendly a a Not friendly . 5 the are food UNK are UNK Boat END friendly
----------------------------------------------
the friendly The 5 . Adriatic , , low 5 is less . Mill CafÃ© The UNK END of friendly are UNK UNK UNK food friendly a UNK the UNK UNK provides UNK UNK the the are . 
 UNK START UNK the a UNK UNK Boat food UNK a a a friendly UNK the the are , UNK UNK UNK Cambridge END a
----------------------------

In [68]:
import re

sequence = [v for i, v in enumerate(values) if i % 64 == 8]

gen_outputs = []
for line in sequence:
    output = re.split('START|END', line)
    cleaned_output = [o for o in output if len(o.strip().split()) > 1]
    gen_outputs.append(cleaned_output[-1])

for output in gen_outputs:
    print()
    print(output)


 UNK the are 5 a low a UNK a a a friendly . 5 the are food average the is Fitzbillies 

 UNK the Golden The UNK UNK I friendly a a Not friendly . 5 the are food UNK are UNK Boat 

 UNK the a UNK UNK Boat food UNK a a a friendly UNK the the are , UNK UNK UNK Cambridge 

 UNK UNK rating UNK UNK UNK food UNK a . a friendly . the the UNK , UNK UNK a Cambridge 

 Located near rating 5 UNK UNK food UNK are . a UNK . English have are , UNK UNK a of Boat range

 Located near The UNK of friendly , UNK are , a food . English the are English UNK UNK a Fitzbillies rating range

 Located near The UNK friendly , The Golden Boat has a food . over the are than food it and are price range

 Located near The Six Bells , The Golden Boat has Indian food . over the average than Â£ the - are price range

 Located near The Six Bells , The Golden Curry has Indian food . In the less than Â£ 20 - 25 price range

 Located near The Six Bells , The Golden Curry has Indian food . In the less than Â£ 20 - 25 price 